# Test the NCF modules under folder [cf_ec2](../cf_ec2) with ml-1m dataset, save the best model

In [1]:
import numpy as np 
import pandas as pd
import keras
from keras import Model
from keras.regularizers import l2
from keras.optimizers import (
    Adam,
    Adamax,
    Adagrad,
    SGD,
    RMSprop
)
from keras.layers import (
    Embedding, 
    Input,
    Flatten, 
    Multiply, 
    Concatenate,
    Dense
)

import sys
sys.path.append('../')
from cf_ec2 import (
    GMF,
    MLP,
    NCF,
    Data,
    evaluation
)

Using TensorFlow backend.


## step 1: load the data

In [2]:
train = pd.read_csv('../data/ml-1m.train.rating',sep='\t',header=None,names=['user','item','rating','event_ts'])
test = pd.read_csv('../data/ml-1m.test.rating',sep='\t',header=None,names=['user','item','rating','event_ts'])

In [3]:
train.head(3)

,user,item,rating,event_ts
0,0,32,4,978824330
1,0,34,4,978824330
2,0,4,5,978824291


In [4]:
test.head(3)

,user,item,rating,event_ts
0,0,25,5,978824351
1,1,133,3,978300174
2,2,207,4,978298504


In [5]:
test.user.nunique(), test.shape

(6040, (6040, 4))

## step 2: prepare the data for ncf model training

In [6]:
dataset = Data(
    train=train,
    test=test,
    col_user='user',
    col_item='item',
    col_rating='rating',
    col_time='event_ts',
    binary=True,
    n_neg=4,
    n_neg_test=100
)
dataset.prepTrainDNN(negSample=True)
dataset.prepTestDNN(group=True)

Method to save python object to disk for later use

```python
import pickle
## pickle data
with open('../metadata/datasetNcf','wb') as fp:
    pickle.dump(dataset, fp)
## pickle data with compression
import bz2
with bz2.BZ2File('datasetNcfSmaller', 'w') as fp:
    pickle.dump(dataset, fp)
    
## unpickle data
with open('../metadata/datasetNcf','rb') as fp:
    dataset2 = pickle.load(fp)
with bz2.BZ2File('../metadata/datasetNcfSmaller', 'r') as fp:
    dataset2 = pickle.load(fp)    
```

In [7]:
len(dataset.users),train.shape

(4970845, (994169, 6))

In [8]:
len(dataset.users_test),test.shape

(610040, (6040, 6))

In [9]:
train.user.nunique(), test.user.nunique()

(6040, 6040)

In [10]:
train.item.nunique(), test.item.nunique()

(3704, 1921)

In [11]:
dataset.interaction_train.head(3)

,user,item_interacted,item_negative
0,0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6..."
1,1,"{15, 22, 31, 34, 35, 42, 43, 52, 53, 54, 55, 5...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,2,"{2, 135, 136, 14, 18, 147, 159, 163, 36, 40, 1...","{0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."


#### prepare the test dataset

In [13]:
newItems = set(dataset.items_test)-set(dataset.items)
idx2del = []
for idx,item in enumerate(dataset.items_test):
    if item in newItems:
        idx2del.append(idx)

length_test_original = len(dataset.users_test)
dataset.users_test = [
    dataset.users_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]
dataset.items_test = [
    dataset.items_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]
dataset.ratings_test = [
    dataset.ratings_test[idx]
    for idx in range(length_test_original) if idx not in idx2del
]

## step 3: create the model architecture

In [14]:
n_users = 6040
n_items = 3704
n_factors_gmf = 32
layers_mlp = [64,32,16,8]
reg_gmf = 0.
reg_layers_mlp = [0.,0.,0.,0.]
learning_rate = 0.01
flg_pretrain = ''
filepath = ''
filepath_gmf_pretrain = ''
filepath_mlp_pretrain = ''
num_epochs = 20
batch_size = 100

ncf = NCF(
    n_users=n_users,
    n_items=n_items,
    n_factors_gmf=n_factors_gmf,
    layers_mlp=layers_mlp,
    reg_gmf=reg_gmf,
    reg_layers_mlp=reg_layers_mlp
)
model = ncf.create_model()
#### compile the model
model.compile(
    optimizer=Adam(lr=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
#### create the callback metrics
filepath="../metadata/ncf/ncf-weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=filepath, 
    verbose=1, 
    save_best_only=True
)
csvlog = keras.callbacks.CSVLogger(
    '../metadata/ncf/ncf_log.csv', 
    separator=',', 
    append=False
)
earlystop = keras.callbacks.EarlyStopping(patience=12)
lrreduce = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.3, 
    patience=4, 
    verbose=1
)

## step 4: train the model

#### define customized metrics

In [15]:
class newMetrics(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
#         print(len(self.validation_data))
#         print(self.validation_data[0][:5])
#         print(self.validation_data[1][:5])        
#         print(self.validation_data[2][:5])
#         print(self.validation_data[3][:5])        
#         X_val, y_val = self.validation_data[0], self.validation_data[1]
        X_val = [self.validation_data[0],self.validation_data[1]]
        y_val = self.validation_data[2]
        y_predict = model.predict(x = X_val)
        logs['val_auc'] = evaluation.auc(y_val, y_predict)

metrics2 = newMetrics()

In [16]:
#### train
hist = model.fit(
    x = [
        np.array(dataset.users),
        np.array(dataset.items)
    ],
    y = np.array(dataset.ratings),
    batch_size=batch_size,
    epochs=num_epochs,
    verbose=2,
    shuffle=True,
    callbacks=[metrics2,checkpoint,csvlog,earlystop,lrreduce],
    validation_data=(
        [
            np.array(dataset.users_test),
            np.array(dataset.items_test)
        ],
        np.array(dataset.ratings_test)
    )
)

/Users/xyin/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4970845 samples, validate on 610038 samples
Epoch 1/20
 - 264s - loss: 0.3348 - accuracy: 0.8501 - val_loss: 0.1916 - val_accuracy: 0.9307

Epoch 00001: val_loss improved from inf to 0.19162, saving model to ../metadata/ncf/ncf-weights-improvement-01-0.1916.hdf5
Epoch 2/20
 - 253s - loss: 0.3059 - accuracy: 0.8648 - val_loss: 0.1869 - val_accuracy: 0.9264

Epoch 00002: val_loss improved from 0.19162 to 0.18687, saving model to ../metadata/ncf/ncf-weights-improvement-02-0.1869.hdf5
Epoch 3/20
 - 260s - loss: 0.2978 - accuracy: 0.8698 - val_loss: 0.1517 - val_accuracy: 0.9387

Epoch 00003: val_loss improved from 0.18687 to 0.15175, saving model to ../metadata/ncf/ncf-weights-improvement-03-0.1517.hdf5
Epoch 4/20
 - 261s - loss: 0.2966 - accuracy: 0.8712 - val_loss: 0.1610 - val_accuracy: 0.9330

Epoch 00004: val_loss did not improve from 0.15175
Epoch 5/20
 - 262s - loss: 0.2980 - accuracy: 0.8716 - val_loss: 0.2356 - val_accuracy: 0.8960

Epoch 00005: val_loss did not improve f

In [ ]:
dataset.users_test[:5], dataset.items_test[:5], dataset.ratings_test[:5], dataset.ratings[:5]

In [ ]:
model.summary()

In [ ]:
hist.history